In [ ]:
!pip install boto3 numpy nibabel matplotlib tensorflow s3fs

In [ ]:
import boto3
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
import os
import tempfile
import asyncio

s3 = boto3.resource('s3')
bucket_name = 'chemocraft-data'
folder_path = 'MICCAI_BraTS2020_TrainingData/'
png_path = 'Theo_PNG/'
bucket = s3.Bucket(bucket_name)

crop_left, crop_right = 20, 10
crop_top, crop_bottom = 30, 30

async def upload_processed_nii(obj):

    filename = obj.key.split('/')[-1]
    print(obj.key)
    file_stream = io.BytesIO(obj.get()['Body'].read())
    
    with tempfile.NamedTemporaryFile(suffix='.nii', delete=True) as temp_file:
        temp_file.write(file_stream.getvalue())
        temp_file.flush()  

        try:
            img = nib.load(temp_file.name)
            data = img.get_fdata()

            start_y = crop_top
            end_y = data.shape[0] - crop_bottom
            start_x = crop_left
            end_x = data.shape[1] - crop_right
            
            if data.size == 0:
                print(f"No data found in {filename}")
                return

            async def up_slice(slice_idx):
                slice_2d = data[:, :, slice_idx]
                cropped_slice = slice_2d[start_y:end_y, start_x:end_x]
                
                buf = io.BytesIO()
                mpimg.imsave(buf, cropped_slice, cmap='gray', format='png')
                buf.seek(0)

                try:
                
                    s3.Object(
                        bucket_name, 
                        f'{png_path}{filename.split('.')[0]}/{slice_idx}.png'
                    ).put(
                        Body=buf,
                        ContentType='image/png'
                    )
                
                except Exception as e:
                    print(f"Error uploading file {filename}: {e}")

            for i in range(data.shape[2]):
                await up_slice(i)

            
        except Exception as e:
            print(f"Error loading file {filename}: {e}")

found_files = False
processed = set([brain.key.split('/')[-2] for brain in bucket.objects.filter(Prefix=png_path)])

for obj in bucket.objects.filter(Prefix=folder_path):
    if obj.key.endswith('.nii'):
        found_files = True
        if obj.key.split('/')[-1].split('.')[0] not in processed or obj.key == 'MICCAI_BraTS2020_TrainingData/BraTS20_Training_058/BraTS20_Training_058_t1.nii':
            await upload_processed_nii(obj)

if not found_files:
    print(f"No .nii files found in the folder {folder_path}")


In [ ]:
import s3fs

bucket_name = 'chemocraft-data'
png_path = 'Theo_PNG/'

# * new library s3fs to more easily navigate dataset
fs = s3fs.S3FileSystem(anon=False)

# * want to rename 1.png -> 001.png to keep in order
# * wasted time trying to make async work
for path, _, files in fs.walk(f's3://{bucket_name}/{png_path}'):
    for file in files:
        new_file = f"{int(file.split('.')[0]):03}.png"
        
        old_path = f"{path}/{file}"
        new_path = f"{path}/{new_file}"
            
        if old_path != new_path:
            print(f"Renaming {old_path} to {new_path}")
            fs.mv(old_path, new_path)

Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/41.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/041.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/42.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/042.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/43.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/043.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/44.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/044.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/45.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/045.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/46.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/046.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/47.png to chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/047.png
Renaming chemocraft-data/Theo_PNG/BraTS20_Training_153_t1ce/48.png to chemocraft-data/Theo_PNG/Br

In [64]:
import io
import random
import numpy as np
import s3fs
from PIL import Image


# * want to create iterator that returns a list of parsed slices from the next folder
# * wasted time trying to animate brain to make sure loaded correctly, but after manually looking at slices it looks right
class S3BrainIterator:
    def __init__(self, bucket_name, prefix):
        self.fs = s3fs.S3FileSystem(anon=False)
        self.bucket_name = bucket_name
        self.prefix = prefix

        self.brain_idx = 0
        self.brain_paths = [] 
        for path in self.fs.ls(f's3://{self.bucket_name}/{self.prefix}'):
            self.brain_paths.append(path+'/')
        random.shuffle(self.brain_paths)

    def __iter__(self):
        self.brain_idx = 0
        random.shuffle(self.brain_paths) # shuffle the dataset after each epoch
        return self

    def __next__(self):
        if self.brain_idx >= len(self.brain_paths):
            raise StopIteration
        
        brain_path = self.brain_paths[self.brain_idx]
        self.brain_idx += 1

        file_paths = self.fs.glob(f'{brain_path}*.png')

        slices = []
        for file_path in file_paths:
            with self.fs.open(file_path, 'rb') as f:
                img = Image.open(io.BytesIO(f.read())).convert('L')
                img_array = np.array(img)
                slices.append(img_array)

        return np.array(slices).astype(np.float32)
        
bi = S3BrainIterator(bucket_name, png_path)

(180, 210)


In [ ]:
from tensorflow.keras import layers, models

def build_generator(latent_dim):
       model = models.Sequential()
       model.add(layers.Dense(155 * 180 * 210, activation="relu", input_dim=latent_dim))
       model.add(layers.Reshape((180, 210, 155)))
       model.add(layers.Conv2DTranspose(155, kernel_size=4, strides=2, padding="same", activation="relu"))
       model.add(layers.Conv2DTranspose(180, kernel_size=4, strides=2, padding="same", activation="relu"))
       model.add(layers.Conv2D(1, kernel_size=3, strides=1, padding="same", activation="tanh"))
       return model

def build_discriminator(img_shape):
       model = models.Sequential()
       model.add(layers.Conv2D(180, kernel_size=4, strides=2, padding="same", input_shape=img_shape, activation="relu"))
       model.add(layers.Conv2D(155, kernel_size=4, strides=2, padding="same", activation="relu"))
       model.add(layers.Flatten())
       model.add(layers.Dense(1, activation="sigmoid"))
       return model

In [ ]:
import tensorflow as tf

bucket_name = 'chemocraft-data'
png_path = 'Theo_PNG/'
bi = S3BrainIterator(bucket_name, png_path)
    
def compile_gan(generator, discriminator, latent_dim):
       discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
       z = layers.Input(shape=(latent_dim,))
       img = generator(z)
       discriminator.trainable = False
       validity = discriminator(img)
       gan = models.Model(z, validity)
       gan.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
       return gan

def train_gan(generator, discriminator, gan, train_slices, epochs, batch_size):
       for epoch in range(epochs):
           for _ in range(len(train_slices) // batch_size):
               # Select random batch of real images
               idx = np.random.randint(0, len(train_slices), batch_size)
               real_slices = np.array([train_slices[i] for i in idx])

               # Generate fake images
               noise = np.random.normal(0, 1, (batch_size, latent_dim))
               fake_slices = generator.predict(noise)

               # Train the discriminator
               d_loss_real = discriminator.train_on_batch(real_slices, np.ones((batch_size, 1)))
               d_loss_fake = discriminator.train_on_batch(fake_slices, np.zeros((batch_size, 1)))
               d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

               # Train the generator
               noise = np.random.normal(0, 1, (batch_size, latent_dim))
               g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

           print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

In [ ]:
from skimage.metrics import structural_similarity as ssim

def generate_3d_image(generator, latent_dim, num_slices=150):
     noise = np.random.normal(0, 1, (num_slices, latent_dim))
     generated_slices = generator.predict(noise)
     generated_3d_image = np.stack(generated_slices, axis=0)  # Shape: (150, 250, 250)
     return generated_3d_image

def evaluate_3d_image(true_image, generated_image):
       for i in range(true_image.shape[0]):  # For each slice
           slice_ssim = ssim(true_image[i], generated_image[i])
           print(f"SSIM for slice {i}: {slice_ssim}")